## Setup Dependencies

In [1]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

### AWS CLI and AWS Python SDK (boto3)

In [2]:
!pip install --disable-pip-version-check -q awscli boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.13.3 requires botocore<1.34.163,>=1.34.70, but you have botocore 1.36.11 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-core 1.1.1 requires scipy<1.13,>=1.5.4, but you have scipy 1.14.1 which is incompatible.
autogluon-features 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omeg

### SageMaker

In [3]:
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug
!pip install --disable-pip-version-check -q sagemaker-experiments

### PyAthena

In [4]:
!pip install --disable-pip-version-check -q PyAthena

### AWS Data Wrangler

In [5]:
!pip install --disable-pip-version-check -q awswrangler

### Zip

In [6]:
!conda install -y zip

Retrieving notices: ...working... done
Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - zip


The following NEW packages will be INSTALLED:

  zip                conda-forge/linux-64::zip-3.0-hd590300_3 

The following packages will be UPDATED:

  amazon-sagemaker-~                    1.0.14-pyhd8ed1ab_0 --> 1.0.16-pyhd8ed1ab_0 
  amazon-sagemaker-~                     3.1.7-pyhd8ed1ab_0 --> 3.1.9-pyhd8ed1ab_0 
  amazon_sagemaker_~                    0.1.13-pyhd8ed1ab_0 --> 0.1.14-pyhd8ed1ab_0 
  aws-glue-sessions                      1.0.7-pyhd8ed1ab_0 --> 1.0.8-pyhd8ed1ab_0 
  ca-certificates                      2024.8.30-hbcca054_0 --> 2025.1.31-hbcca054_0 
  certifi                            2024.8.30-pyhd8ed1ab_0 --> 2024.12.14-pyhd8ed1ab_0 
  conda                              24.9.2-py311h38be061_0 --> 25.1.1-py311h38be061_0 
  fastapi                              0.115

### Matplotlib

In [7]:
!pip install --disable-pip-version-check -q matplotlib

### Seaborn

In [8]:
!pip install --disable-pip-version-check -q seaborn

## Data lake Setup

In [9]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = 'flightfinalapproachanomalydetection'#'sagemaker-us-east-1-817854913925' #sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

print("Bucket name: {}".format(bucket))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Bucket name: flightfinalapproachanomalydetection


### Verify Bucket Creation

In [10]:
from botocore.client import ClientError

response = None
try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e: 
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'CGN8JJ5XWBHKSV11', 'HostId': 'X8S460Q5aaZvyM/cl57Y89hLlAM3J45RmgJzRorn7IpuoU5EAfTX9EkDSDUE1LJTvtDKo7q8Gn0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'X8S460Q5aaZvyM/cl57Y89hLlAM3J45RmgJzRorn7IpuoU5EAfTX9EkDSDUE1LJTvtDKo7q8Gn0=', 'x-amz-request-id': 'CGN8JJ5XWBHKSV11', 'date': 'Sat, 01 Feb 2025 22:53:39 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [11]:
role = sagemaker.get_execution_role()
region = boto3.Session().region_name 
account_id = boto3.client(service_name="sagemaker", region_name=region)

In [12]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/flightfinalapproachanomalydetection/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

/tmp/ipykernel_1270/88463448.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


## Create Athena Database Scheme
### Import PyAthena

In [13]:
from pyathena import connect

In [14]:
table_name_csv = "flight_csv_data"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

### Create Athena Database

In [15]:
database_name = "flightdata_db"

In [16]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [17]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [18]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS flightdata_db


In [19]:
cursor = conn.cursor()
cursor.execute(statement)
print("Database created successfully")


Database created successfully


In [20]:
import pandas as pd

statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1270/2310636415.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,assignment2_aws
1,default
2,dsoaws
3,flightdata_db
4,sagemaker_featurestore


## Register CSV with Athena

In [21]:
drop_statement = "DROP TABLE flightdata_db.flight_csv_data"
cursor = conn.cursor()
cursor.execute(drop_statement)
print("Table dropped successfully.")

Table dropped successfully.


In [22]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS flightdata_db.flight_csv_data (
        sample_id int,
        timestep int,
        aileron_pos_lh_deg float,
        aileron_pos_rh_deg float,
        corrected_angle_of_attack_deg float,
        baro_correct_alt_lsp_ft float,
        computed_airspeed_lsp_knots float,
        selected_course_deg float,
        drift_angle_deg float,
        elevator_pos_left_deg float,
        te_flap_pos_disc float,
        glideslope_dev_perc float,
        selected_heading_deg float,
        localizer_dev_perc float,
        core_speed_avg_perc float,
        total_pressure_lsp_millibar float,
        pitch_angle_lsp_deg float,
        roll_angle_lsp_deg float,
        rudder_pos_deg float,
        true_heading_lsp_deg float,
        vertical_accel_g float,
        wind_speed_knots float,
        label int
)
STORED AS TEXTFILE
LOCATION 's3://flightfinalapproachanomalydetection/csv/'
TBLPROPERTIES (
    'skip.header.line.count'='1',
    'field.delim'=',',
    'compressionType'='none'
)"""

In [23]:
cursor = conn.cursor()
cursor.execute(statement)

In [24]:
import pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_1270/1322682284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [25]:
statement = "SHOW TABLES IN {}".format(database_name)
cursor = conn.cursor()
cursor.execute(statement)

In [26]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_csv_passed = True
else:
    ingest_create_athena_table_csv_passed = False
print(ingest_create_athena_table_csv_passed)

False


In [27]:
print(database_name)
print(table_name_csv)

flightdata_db
flight_csv_data


In [28]:
statement = """SELECT sample_id, aileron_pos_lh_deg, timestep, label FROM {}.{}
    LIMIT 10""".format(
    database_name, table_name_csv
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT sample_id, aileron_pos_lh_deg, timestep, label FROM flightdata_db.flight_csv_data
    LIMIT 10


/tmp/ipykernel_188/1784874562.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,sample_id,aileron_pos_lh_deg,timestep,label
0,0,81.261190,0,0
1,0,79.604095,1,0
2,0,81.302110,2,0
3,0,82.345470,3,0
4,0,81.874930,4,0


In [29]:
statement = """SELECT label, COUNT(*) AS count
    FROM {}.{}
    GROUP BY label
    ORDER BY label""".format(
    database_name, table_name_csv
)


df = pd.read_sql(statement, conn)
df.head(10)

/tmp/ipykernel_188/624500045.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,label,count
0,0,14346080
1,1,1122080
2,2,353120
3,3,152640


The classes are significantly imbalanced, with 14 million nominal examples, 1 million speed high examples, 350,000 path high examples and 152,000 flaps late setting examples. It should be noted that this is time series data, so the number of individual flight recordings is a fraction of this.

### Check for Duplicates

In [33]:
statement = """SELECT *, COUNT(*)
    FROM {}.{}
    GROUP BY sample_id, aileron_pos_lh_deg, timestep, aileron_pos_rh_deg, corrected_angle_of_attack_deg, baro_correct_alt_lsp_ft, computed_airspeed_lsp_knots,
    selected_course_deg, drift_angle_deg, elevator_pos_left_deg, te_flap_pos_disc, glideslope_dev_perc, selected_heading_deg, localizer_dev_perc, core_speed_avg_perc,
    total_pressure_lsp_millibar, pitch_angle_lsp_deg, roll_angle_lsp_deg, rudder_pos_deg, true_heading_lsp_deg, vertical_accel_g, wind_speed_knots, label
    HAVING COUNT(*) > 1""".format(
    database_name, table_name_csv
)

df = pd.read_sql(statement, conn)
df.head(10)


/tmp/ipykernel_188/2534158001.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,sample_id,timestep,aileron_pos_lh_deg,aileron_pos_rh_deg,corrected_angle_of_attack_deg,baro_correct_alt_lsp_ft,computed_airspeed_lsp_knots,selected_course_deg,drift_angle_deg,elevator_pos_left_deg,...,core_speed_avg_perc,total_pressure_lsp_millibar,pitch_angle_lsp_deg,roll_angle_lsp_deg,rudder_pos_deg,true_heading_lsp_deg,vertical_accel_g,wind_speed_knots,label,_col23


#### Check for Null or Missing Values

In [24]:
!pwd

/home/sagemaker-user/FlightFinalApproachAnomalyDetection


In [25]:
import pandas as pd
df = pd.read_csv("/home/sagemaker-user/FlightFinalApproachAnomalyDetection/training_flight_data.csv")

In [27]:
df.head()

,SampleID,TimeStep,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,...,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Label
0,0,0,81.261190,82.652336,-8.111792,1969.6174,155.57140,-2.109358,-0.692778,-4.952854,...,0.006664,70.74118,985.42550,-3.662262,0.785912,-0.390141,-1.081870,0.972379,12.625183,0.0
1,0,1,79.604095,81.015700,-7.644611,1955.6995,154.51205,-2.109358,-0.867216,-5.198349,...,0.007448,70.71775,985.52030,-3.665276,0.046774,-0.756234,-0.704820,0.770077,11.893839,0.0
2,0,2,81.302110,80.770200,-7.552573,1940.0267,153.32867,-2.109358,-1.424093,-4.830105,...,0.009604,70.70276,985.41650,-3.940319,0.804820,-1.325632,-0.240446,0.543937,12.559112,0.0
3,0,3,82.345470,83.900276,-8.395265,1924.5493,150.88818,-2.109358,-1.141912,-4.625526,...,0.009408,70.74159,984.81710,-4.275129,1.077102,-0.326884,-0.191627,1.062817,10.542998,0.0
4,0,4,81.874930,82.754620,-7.854284,1905.3670,150.69461,-2.109358,-0.724660,-4.400490,...,0.009408,70.57045,985.23065,-4.241483,1.654806,0.129545,-0.528425,0.867628,9.713539,0.0


In [26]:

df.columns = ['sample_id', 'time_step', 'aileron_pos_lh_deg', 'aileron_pos_rh_deg', 'corrected_angle_of_attack_deg', 'baro_correct_alt_lsp_ft', 'computed_airspeed_lsp_knots',
    'selected_course_deg', 'drift_angle_deg', 'elevator_pos_left_deg', 'te_flap_pos_disc', 'glideslope_dev_perc', 'selected_heading_deg', 'localizer_dev_perc', 'core_speed_avg_perc',
    'total_pressure_lsp_millibar', 'pitch_angle_lsp_deg', 'roll_angle_lsp_deg', 'rudder_pos_deg', 'true_heading_lsp_deg', 'vertical_accel_g', 'wind_speed_knots', 'label']

In [29]:
missing_count = df.isnull().sum() 

print(df.isnull().sum())

sample_id                        0
time_step                        0
aileron_pos_lh_deg               0
aileron_pos_rh_deg               0
corrected_angle_of_attack_deg    0
baro_correct_alt_lsp_ft          0
computed_airspeed_lsp_knots      0
selected_course_deg              0
drift_angle_deg                  0
elevator_pos_left_deg            0
te_flap_pos_disc                 0
glideslope_dev_perc              0
selected_heading_deg             0
localizer_dev_perc               0
core_speed_avg_perc              0
total_pressure_lsp_millibar      0
pitch_angle_lsp_deg              0
roll_angle_lsp_deg               0
rudder_pos_deg                   0
true_heading_lsp_deg             0
vertical_accel_g                 0
wind_speed_knots                 0
label                            0
dtype: int64


#### Boxplots of Data Distribution

In [ ]:
import matplotlib.pyplot as plt

df.plot(kind='box', subplots=True, layout=(5, 5), figsize=(15, 15))
plt.show()

### Variable Correlation

In [ ]:
import seaborn as sns

corr_matrix = df.corr()
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', )

plt.show()

In [27]:

df["record_identifier"] = df["sample_id"].astype(str) + "_" + df["time_step"].astype(str)

df["time_step"] = df["time_step"].astype(float)
df.tail()

,sample_id,time_step,aileron_pos_lh_deg,aileron_pos_rh_deg,corrected_angle_of_attack_deg,baro_correct_alt_lsp_ft,computed_airspeed_lsp_knots,selected_course_deg,drift_angle_deg,elevator_pos_left_deg,...,core_speed_avg_perc,total_pressure_lsp_millibar,pitch_angle_lsp_deg,roll_angle_lsp_deg,rudder_pos_deg,true_heading_lsp_deg,vertical_accel_g,wind_speed_knots,label,record_identifier
6389915,99835,155.0,88.114624,82.222720,6.874017,629.22797,105.47570,34.98022,-1.176979,-3.316216,...,63.319550,1017.93130,4.400115,0.204086,-0.666766,29.992224,1.082715,3.502982,3.0,99835_155
6389916,99835,156.0,81.956764,86.375694,6.319379,623.89624,104.67262,34.98022,-1.019591,1.552792,...,61.987160,1017.81550,4.322569,-0.970712,-0.622764,29.740330,1.058380,3.245408,3.0,99835_156
6389917,99835,157.0,86.396150,86.027910,4.170040,624.05475,101.82832,34.98022,-1.167051,1.961948,...,61.082344,1016.82410,3.697519,0.035246,-0.695083,29.869590,1.056430,3.285423,3.0,99835_157
6389918,99835,158.0,85.577835,85.270966,1.343325,624.10240,99.21132,34.98022,-0.159543,10.063316,...,58.991150,1015.96124,2.390611,0.591246,-0.648484,29.010420,0.966415,3.440334,3.0,99835_158
6389919,99835,159.0,84.636765,86.682560,1.255068,624.51654,97.39182,34.98022,0.457792,9.797363,...,56.885094,1015.51624,2.680404,0.820136,-0.642458,28.177433,1.043380,2.406222,3.0,99835_159


## Feature Store

In [29]:
from sagemaker.feature_store.feature_group import FeatureGroup
from time import gmtime, strftime, sleep
from sagemaker.session import Session

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

record_identifer_feature_name = 'record_identifier'
event_time_feature_name = 'time_step'

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

flight_feature_group_name = "flightfeature-group-" + strftime("%d-%H-%M-%S", gmtime())
flight_feature_group = FeatureGroup(name=flight_feature_group_name, sagemaker_session=feature_store_session
                                     )

In [42]:
df.head()

,sample_id,time_step,aileron_pos_lh_deg,aileron_pos_rh_deg,corrected_angle_of_attack_deg,baro_correct_alt_lsp_ft,computed_airspeed_lsp_knots,selected_course_deg,drift_angle_deg,elevator_pos_left_deg,...,core_speed_avg_perc,total_pressure_lsp_millibar,pitch_angle_lsp_deg,roll_angle_lsp_deg,rudder_pos_deg,true_heading_lsp_deg,vertical_accel_g,wind_speed_knots,label,record_identifier
0,0,0.0,81.261190,82.652336,-8.111792,1969.6174,155.57140,-2.109358,-0.692778,-4.952854,...,70.74118,985.42550,-3.662262,0.785912,-0.390141,-1.081870,0.972379,12.625183,0.0,0_0
1,0,1.0,79.604095,81.015700,-7.644611,1955.6995,154.51205,-2.109358,-0.867216,-5.198349,...,70.71775,985.52030,-3.665276,0.046774,-0.756234,-0.704820,0.770077,11.893839,0.0,0_1
2,0,2.0,81.302110,80.770200,-7.552573,1940.0267,153.32867,-2.109358,-1.424093,-4.830105,...,70.70276,985.41650,-3.940319,0.804820,-1.325632,-0.240446,0.543937,12.559112,0.0,0_2
3,0,3.0,82.345470,83.900276,-8.395265,1924.5493,150.88818,-2.109358,-1.141912,-4.625526,...,70.74159,984.81710,-4.275129,1.077102,-0.326884,-0.191627,1.062817,10.542998,0.0,0_3
4,0,4.0,81.874930,82.754620,-7.854284,1905.3670,150.69461,-2.109358,-0.724660,-4.400490,...,70.57045,985.23065,-4.241483,1.654806,0.129545,-0.528425,0.867628,9.713539,0.0,0_4


In [43]:
flight_feature_group.load_feature_definitions(data_frame=df)

[FeatureDefinition(feature_name='sample_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='time_step', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='aileron_pos_lh_deg', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='aileron_pos_rh_deg', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='corrected_angle_of_attack_deg', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='baro_correct_alt_lsp_ft', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='computed_airspeed_lsp_knots', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='selected_course_deg', feat

### Creation of FeatureGroups in SageMaker FeatureStore

In [44]:
import time

def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


flight_feature_group.create(
    s3_uri=f"s3://{bucket}/feature_group",
    record_identifier_name=record_identifer_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
)


wait_for_feature_group_creation_complete(feature_group=flight_feature_group)

ResourceInUse: An error occurred (ResourceInUse) when calling the CreateFeatureGroup operation: Resource Already Exists: FeatureGroup with name flightfeature-group-01-17-17-38 already exists. Choose a different name.

In [45]:
flight_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:817854913925:feature-group/flightfeature-group-01-17-17-38',
 'FeatureGroupName': 'flightfeature-group-01-17-17-38',
 'RecordIdentifierFeatureName': 'record_identifier',
 'EventTimeFeatureName': 'time_step',
 'FeatureDefinitions': [{'FeatureName': 'sample_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'time_step', 'FeatureType': 'Fractional'},
  {'FeatureName': 'aileron_pos_lh_deg', 'FeatureType': 'Fractional'},
  {'FeatureName': 'aileron_pos_rh_deg', 'FeatureType': 'Fractional'},
  {'FeatureName': 'corrected_angle_of_attack_deg',
   'FeatureType': 'Fractional'},
  {'FeatureName': 'baro_correct_alt_lsp_ft', 'FeatureType': 'Fractional'},
  {'FeatureName': 'computed_airspeed_lsp_knots', 'FeatureType': 'Fractional'},
  {'FeatureName': 'selected_course_deg', 'FeatureType': 'Fractional'},
  {'FeatureName': 'drift_angle_deg', 'FeatureType': 'Fractional'},
  {'FeatureName': 'elevator_pos_left_deg', 'FeatureType': 'Fractional'},
  {'Featu

In [46]:
sagemaker_client.list_feature_groups() 

{'FeatureGroupSummaries': [{'FeatureGroupName': 'neighborhood-feature-group-26-05-02-41',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:817854913925:feature-group/neighborhood-feature-group-26-05-02-41',
   'CreationTime': datetime.datetime(2025, 1, 26, 5, 2, 52, 140000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'neighborhood-feature-group-26-04-11-14',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:817854913925:feature-group/neighborhood-feature-group-26-04-11-14',
   'CreationTime': datetime.datetime(2025, 1, 26, 4, 11, 18, 797000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'flightfeature-group-31-22-05-51',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:817854913925:feature-group/flightfeature-group-31-22-05-51',
   'CreationTime': datetime.datetime(2025, 1, 31, 22, 5, 52, 653000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreS

In [ ]:
flight_feature_group.ingest(data_frame=df, max_workers=8, wait=True)

In [30]:
feats_to_quer = ['0_0','0_150','99835_155']

for val in feats_to_quer:
    record_identifier_value = val
    
    response = featurestore_runtime.get_record(
        FeatureGroupName='flightfeature-group-01-17-17-38',
        RecordIdentifierValueAsString=record_identifier_value,
    )
    
    if 'Record' in response and response['Record']:
        print("Record",val,"found:")
        for feature in response['Record']:
            print(f"{feature['FeatureName']}: {feature['ValueAsString']}")
        print("\n\n")
    else:
        print("Record",val,"not found.")
        print("\n\n")


Record 0_0 found:
sample_id: 0
time_step: 0.0
aileron_pos_lh_deg: 81.26119
aileron_pos_rh_deg: 82.652336
corrected_angle_of_attack_deg: -8.111792
baro_correct_alt_lsp_ft: 1969.6174
computed_airspeed_lsp_knots: 155.5714
selected_course_deg: -2.1093580000000003
drift_angle_deg: -0.6927779000000001
elevator_pos_left_deg: -4.952854
te_flap_pos_disc: 3065.0
glideslope_dev_perc: 0.01794
selected_heading_deg: -2.1093580000000003
localizer_dev_perc: 0.006664
core_speed_avg_perc: 70.74118
total_pressure_lsp_millibar: 985.4255
pitch_angle_lsp_deg: -3.6622615
roll_angle_lsp_deg: 0.7859119999999999
rudder_pos_deg: -0.39014083
true_heading_lsp_deg: -1.0818703
vertical_accel_g: 0.9723786
wind_speed_knots: 12.625183
label: 0.0
record_identifier: 0_0



Record 0_150 found:
sample_id: 0
time_step: 150.0
aileron_pos_lh_deg: 77.41508499999999
aileron_pos_rh_deg: 81.30211
corrected_angle_of_attack_deg: -4.732262599999999
baro_correct_alt_lsp_ft: 422.82614
computed_airspeed_lsp_knots: 121.19918
selected_co

In [38]:
df.head()

,sample_id,time_step,aileron_pos_lh_deg,aileron_pos_rh_deg,corrected_angle_of_attack_deg,baro_correct_alt_lsp_ft,computed_airspeed_lsp_knots,selected_course_deg,drift_angle_deg,elevator_pos_left_deg,...,core_speed_avg_perc,total_pressure_lsp_millibar,pitch_angle_lsp_deg,roll_angle_lsp_deg,rudder_pos_deg,true_heading_lsp_deg,vertical_accel_g,wind_speed_knots,label,record_identifier
0,0,0.0,81.261190,82.652336,-8.111792,1969.6174,155.57140,-2.109358,-0.692778,-4.952854,...,70.74118,985.42550,-3.662262,0.785912,-0.390141,-1.081870,0.972379,12.625183,0.0,0_0
1,0,1.0,79.604095,81.015700,-7.644611,1955.6995,154.51205,-2.109358,-0.867216,-5.198349,...,70.71775,985.52030,-3.665276,0.046774,-0.756234,-0.704820,0.770077,11.893839,0.0,0_1
2,0,2.0,81.302110,80.770200,-7.552573,1940.0267,153.32867,-2.109358,-1.424093,-4.830105,...,70.70276,985.41650,-3.940319,0.804820,-1.325632,-0.240446,0.543937,12.559112,0.0,0_2
3,0,3.0,82.345470,83.900276,-8.395265,1924.5493,150.88818,-2.109358,-1.141912,-4.625526,...,70.74159,984.81710,-4.275129,1.077102,-0.326884,-0.191627,1.062817,10.542998,0.0,0_3
4,0,4.0,81.874930,82.754620,-7.854284,1905.3670,150.69461,-2.109358,-0.724660,-4.400490,...,70.57045,985.23065,-4.241483,1.654806,0.129545,-0.528425,0.867628,9.713539,0.0,0_4
